In [ ]:
# IGNORE THIS CELL WHICH CUSTOMIZES LAYOUT AND STYLING OF THE NOTEBOOK !
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings = lambda *a, **kw: None

# Exercise 3_psyplot Mesh plots (psyplot)
prepared by A. Lauber

Here we learn how to plot data from an unstructured grid like the [ICON](https://code.mpimet.mpg.de/projects/iconpublic) grid as mesh grid.
There is the option to interpolate the data to a regular grid or to use the library [psyplot](https://psyplot.github.io), which was developed for plotting data on the unstructured grid.

Advantages of plotting on the unstructured grid:
- Shows the real output and no interpolation
- No preprocessing of the data necessary

Disadvantages of plotting on the unstructured grid:
- It can be quite slow depending on the size of your netCDF file
- Not all features are available yet (psyplot is still under development)


## Import libraries

In [ ]:
import xarray as xr
import numpy as np
import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
import cmcrameri.cm as cmc
import urllib.request

## Download data

We will use a netCDF file created with an ICON run by Nadja Omanovic.
It was reduced to the 2m temperature over Switzerland to save space. The file is saved on the FTP server from IAC.

In [ ]:
ftp_pyvis = "ftp://iacftp.ethz.ch/pub_read/alauber/pyvis/"
filename = "my_exp1_atm_3d_ml_20180921T000000Z_t2m.nc"
urllib.request.urlretrieve(ftp_pyvis+filename, filename)
filename = "rmp_my_exp1_atm_3d_ml_20180921T000000Z.nc"
urllib.request.urlretrieve(ftp_pyvis+filename, filename)

## Now have a look into the data

In [ ]:
ds = xr.open_dataset("my_exp1_atm_3d_ml_20180921T000000Z_t2m.nc")
ds

## Plot data with pcolormesh
Why does it not work?

In [ ]:
# Get data
lon, lat, temp = ds.clon, ds.clat, ds.t_2m[:,:,:]

# Code here


Answer: pcolormesh cannot handle data on an unstructered grid.

## Plot data with pcolormesh using remapped data

Have a look into the remapped data first:

In [ ]:
ds_rmp = xr.open_dataset("rmp_my_exp1_atm_3d_ml_20180921T000000Z.nc")
ds_rmp

### Exercise
 * Plot the 2m temperature over Switzerland
 * Use the projection=ccrs.Robinson() (don't forget to transform the data)
 * Add the borders of Switzerland
 * Cut off data outside of Switzerland (5.8<lon<10.7, 45.5<lat<48)
 * Add a colorbar
 * Set limits to the colorbar
 * Use the colormap cmc.nuuk

In [ ]:
# get data
lon, lat, temp = ds_rmp.lon, ds_rmp.lat, ds_rmp.t_2m[0,0,:,:]

# Code for plotting here


## Solution

In [ ]:
# get data
lon, lat, temp = ds_rmp.lon, ds_rmp.lat, ds_rmp.t_2m[0,0,:,:]

ax = plt.axes(projection=ccrs.Robinson())

h = ax.pcolormesh(lon,lat,temp, transform=ccrs.PlateCarree(), cmap = cmc.nuuk,
             vmin=260,vmax=300)

ax.add_feature(cartopy.feature.BORDERS)

ax.set_extent([5.8, 10.7, 45.5, 48])
plt.colorbar(h)
plt.show()

## Let's do the same now on the original grid
We will now get started with [psyplot](https://psyplot.github.io). You can have a look at the website.

First we need to import the psyplot library:

In [ ]:
import psyplot.project as psy
psy.rcParams['auto_show'] = True

Let's now take the file with the original ICON grid and plot it.

## Load data with psyplot

In [ ]:
# Load dataset with psyplot
ds_icon = psy.open_dataset("my_exp1_atm_3d_ml_20180921T000000Z_t2m.nc")
ds_icon

## Plot data with psyplot

In [ ]:
plot_icon = ds_icon.psy.plot.mapplot(name='t_2m')

Guess we can do that nicer....

Checkout the [available formatoptions](https://psyplot.github.io/psy-maps/generated/psyplot.project.plot.mapplot.html) and try to make the same plot as before.


### Exercise
 * Plot the 2m temperature over Switzerland
 * Use the projection=ccrs.Robinson() (don't forget to transform the data)
 * Add the borders of Switzerland
 * Cut off data outside of Switzerland (5.8<lon<10.7, 45.5<lat<48)
 * Set limits to the colorbar
 * Use the colormap cmc.nuuk
 
 (Hints: map_extent, bounds={'method':'..','vmin':'..'}

In [ ]:
plot_icon = ds_icon.psy.plot.mapplot(name='t_2m',
                                     projection=ccrs.Robinson(),
                                     map_extent=[5.8, 10.7, 45.5, 48],
                                     cmap=cmc.nuuk,
                                     bounds={'method':'minmax','vmin':270,'vmax':300})

In [ ]:
#Checkout https://github.com/C2SM/iconarray
import iconarray
%config InlineBackend.close_figures = False

In [ ]:
# Now you do the same plot but on the unstructured grid
ds_icon = psy.open_dataset("my_exp1_atm_3d_ml_20180921T000000Z_t2m.nc")
lonmin = 5.8
lonmax = 10.7
latmin = 45.5
latmax = 48.0
plot_icon = ds_icon.psy.plot.mapplot(
    name = 't_2m',
    t = 0,
    projection = 'robin',
    bounds = {'method': 'minmax', 'vmin':260, 'vmax':300},
    map_extent = [lonmin, lonmax, latmin, latmax],
    title = '2m temperature',
    cmap = cmc.nuuk,
    xgrid=False,
    ygrid=False)

In [ ]:
plot_icon.update(lakes=False)